# Initialization

Test notebook for the C-MAPPS benchmark. Test the Tunable Model class architecture

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

Using TensorFlow backend.


# Create models

Create different types of models to be tested with the Tunable Model class

In [2]:
#Clear the previous tensorflow graph
K.clear_session()

FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(100, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

# Load the data

In [3]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

windowSize = 20
stride = 2
constRUL = 125


dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, 'keras', window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
#tModel.changeDataset(2)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData()

Loading data for dataset 1 with window_size of 20, stride of 2 and constRUL of 125. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(9390, 280)
(9390, 1)
Testing data (X, y)
(100, 280)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.62199313 -0.07920059 -0.42905692 ..., -0.4         0.36        0.52472609]
 [-0.25773196 -0.16358253 -0.27612574 ..., -0.4         0.184       0.39798638]
 [-0.24398625 -0.41722181 -0.19031436 ..., -0.4         0.312       0.36955878]
 [-0.16838488  0.05008636 -0.54927782 ...,  0.          0.088       0.50370151]
 [-0.41580756 -0.0160375  -0.67544605 ..., -0.8         0.328       0.11637548]]
[[ 125.]
 [ 125.]
 [ 125.]
 [ 125.]
 [ 125.]]
Testing data (X, y)
[[-0.12714777 -0.19121638 -0.54715378 ..., -0.2         0.008       0.32691738]
 [ 0.01718213 -0.37132988 -0.03101105 ..., -0.6         0.008       0.05034054]
 [ 0.04467354 -0.10584752 -0.09005947 ...,  0.2         0.2         0.08646728]
 [-0.580756

In [4]:
#tModel.printData(printTop=False)
#tModel.windowSize = 20
#tModel.changeDataset(2)
#tModel.loadData(verbose=1, crossValRatio=0.1)
#tModel.printData(printTop=False)
#tModel.getModelDescription()
#ruldcnn = RULCNNModel(tModel.windowSize, len(selected_features))
#ruldcnn.summary()
tModel.trimmedRUL_train

array([ 67, 162,  54,  64, 144,  63, 134,  25,  76,  97, 115,  45,  38,
        55,  82,  84, 151,  70,  33, 109,  70,  77,  43,  22, 105,  74,
        31,  40,  38,  69, 109,  66,  75,  70,  56,  33,  45,  69,   3,
        63,  91,  71,  82,  67,  33, 131,  89, 106,  90,  73,  88,  88,
        70, 132,  68, 150,  12,  22, 106,  47,  60,  55,  49, 158,  28,
        77, 188,  74, 237,  12,  83,  88,  88,  41, 104,  85,  29, 106,
        74,  60, 115,  89, 168, 142,  63, 153,  53,  88,  92,  29,  10,
       216,  30, 133, 158, 211,  77,  31,  60,  75])

# Train the model and test some Tunable Model functionalities

In [5]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
#tModel.setCurrentModel('RULmodel_SN_1')
#tModel.getModelDescription()
tModel.epochs = 100
tModel.trainModel(learningRateScheduler=lrate, verbose=1)

Epoch 1/100
9390/9390 [==============================] - 1s 61us/step - loss: 8452.2338 - mean_squared_error: 8452.2338
Epoch 2/100
9390/9390 [==============================] - 0s 11us/step - loss: 7438.8043 - mean_squared_error: 7438.8043
Epoch 3/100
9390/9390 [==============================] - 0s 14us/step - loss: 5298.3732 - mean_squared_error: 5298.3732
Epoch 4/100
9390/9390 [==============================] - 0s 12us/step - loss: 3133.0476 - mean_squared_error: 3133.0476
Epoch 5/100
9390/9390 [==============================] - 0s 11us/step - loss: 1972.9776 - mean_squared_error: 1972.9776
Epoch 6/100
9390/9390 [==============================] - 0s 11us/step - loss: 1571.2656 - mean_squared_error: 1571.2656
Epoch 7/100
9390/9390 [==============================] - 0s 11us/step - loss: 1346.6201 - mean_squared_error: 1346.6201
Epoch 8/100
9390/9390 [==============================] - 0s 12us/step - loss: 1203.8469 - mean_squared_error: 1203.8469
Epoch 9/100
9390/9390 [=================

Epoch 69/100
9390/9390 [==============================] - 0s 27us/step - loss: 748.0748 - mean_squared_error: 748.0748
Epoch 70/100
9390/9390 [==============================] - 0s 18us/step - loss: 726.7249 - mean_squared_error: 726.7249
Epoch 71/100
9390/9390 [==============================] - 0s 9us/step - loss: 745.8784 - mean_squared_error: 745.8784
Epoch 72/100
9390/9390 [==============================] - 0s 10us/step - loss: 738.2643 - mean_squared_error: 738.2643
Epoch 73/100
9390/9390 [==============================] - 0s 38us/step - loss: 743.3480 - mean_squared_error: 743.3480
Epoch 74/100
9390/9390 [==============================] - 0s 11us/step - loss: 731.1391 - mean_squared_error: 731.1391
Epoch 75/100
9390/9390 [==============================] - 0s 11us/step - loss: 752.1247 - mean_squared_error: 752.1247
Epoch 76/100
9390/9390 [==============================] - 0s 10us/step - loss: 728.3808 - mean_squared_error: 728.3808
Epoch 77/100
9390/9390 [=========================

In [6]:
tModel.evaluateModel(["rhs"], round=2, scalingFactor=20)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
for prediction in tModel.y_pred:
    print('Engine {}, RUL {}, Rounded RUL {}, Real RUL {}'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1]))
    i = i+1

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 286us/step
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
scores
Engine 1, RUL [ 107.98149109], Rounded RUL 107.0, Real RUL [ 112.]
Engine 2, RUL [ 107.21675873], Rounded RUL 107.0, Real RUL [ 98.]
Engine 3, RUL [ 47.48054504], Rounded RUL 47.0, Real RUL [ 69.]
Engine 4, RUL [ 75.09249115], Rounded RUL 75.0, Real RUL [ 82.]
Engine 5, RUL [ 87.44521332], Rounded RUL 87.0, Real RUL [ 91.]
Engine 6, RUL [ 93.69506836], Rounded RUL 93.0, Real RUL [ 93.]
Engine 7, RUL [ 115.18941498], Rounded RUL 115.0, Real RUL [ 91.]
Engine 8, RUL [ 82.21271515], Rounded RUL 82.0, Real RUL [ 95.]
Engine 9, RUL [ 100.89044952], Rounded RUL 100.0, Real RUL [ 111.]
Engine 10, RUL [ 80.68328094], Rounded RUL 80.

In [ ]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"], crossValidation=True)
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse

In [ ]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    
    maxCycle = X_test2.shape[0]
    faultCycle = tModel.y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()

In [ ]:
plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, stride, constRUL)